In [1]:
pip install --upgrade torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [2]:
print("### RUNNING CELL 1: TRAINING ###")

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset, random_split
import pandas as pd
import os
from PIL import Image
import time

# --------------------------------------------------------------------------
# Step 1: Setup and Path Configuration
# --------------------------------------------------------------------------
print("Step 1: Configuring paths...")

# !!! IMPORTANT !!!
# Update this base path to match your project's main folder.
BASE_PATH = r'C:\Users\harkp\Desktop\HV-AI-2025\HV-AI-2025'

DATA_DIR = os.path.join(BASE_PATH, 'labeled_data', 'images')
CSV_PATH = os.path.join(BASE_PATH, 'labeled_data', 'labeled_data.csv')
MODEL_SAVE_PATH = 'phase1_baseline_model.pth'

print(f"Image data directory: {DATA_DIR}")
print(f"Labels CSV path: {CSV_PATH}")
print("-" * 50)

# --------------------------------------------------------------------------
# Step 2: Custom Dataset and DataLoaders
# --------------------------------------------------------------------------
print("Step 2: Defining Custom Dataset and DataLoaders...")

class LabeledDataset(Dataset):
    """Custom Dataset for reading image paths and labels from a CSV."""
    def __init__(self, csv_path, img_dir, transform=None):
        self.labels_df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform
        self.class_names = sorted(self.labels_df['label'].unique())
        self.class_to_idx = {name: i for i, name in enumerate(self.class_names)}
        self.idx_to_class = {i: name for name, i in self.class_to_idx.items()}

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = self.labels_df.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        label_name = self.labels_df.iloc[idx, 1]
        label_idx = self.class_to_idx[label_name]
        if self.transform:
            image = self.transform(image)
        return image, label_idx

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

full_dataset = LabeledDataset(csv_path=CSV_PATH, img_dir=DATA_DIR)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_subset, val_subset = random_split(full_dataset, [train_size, val_size])

class TransformedDataset(Dataset):
    def __init__(self, subset, transform):
        self.subset = subset
        self.transform = transform
    def __getitem__(self, index):
        x, y = self.subset[index]
        return self.transform(x), y
    def __len__(self):
        return len(self.subset)

train_dataset = TransformedDataset(train_subset, data_transforms['train'])
val_dataset = TransformedDataset(val_subset, data_transforms['val'])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
num_classes = len(full_dataset.class_names)
print(f"Dataset ready. Found {num_classes} classes.")
print("-" * 50)

# --------------------------------------------------------------------------
# Step 3: Model Definition (ResNet-18)
# --------------------------------------------------------------------------
print("Step 3: Defining the ResNet-18 model...")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = models.resnet18(weights='IMAGENET1K_V1')
for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
model = model.to(device)
print("Model definition complete.")
print("-" * 50)

# --------------------------------------------------------------------------
# Step 4: Training the Model
# --------------------------------------------------------------------------
print("Step 4: Starting model training...")
start_time = time.time()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
num_epochs = 10
best_val_acc = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)

    model.eval()
    val_loss = 0.0
    corrects = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            val_loss += loss.item() * inputs.size(0)
            corrects += torch.sum(preds == labels.data)
    epoch_val_loss = val_loss / len(val_loader.dataset)
    epoch_val_acc = corrects.double() / len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {epoch_loss:.4f} | Val Loss: {epoch_val_loss:.4f} | Val Acc: {epoch_val_acc:.4f}")

    if epoch_val_acc > best_val_acc:
        best_val_acc = epoch_val_acc
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"  -> Validation accuracy improved. Saving model to {MODEL_SAVE_PATH}")

training_time = time.time() - start_time
print(f"\nTraining complete in {training_time // 60:.0f}m {training_time % 60:.0f}s")
print(f"Best validation accuracy: {best_val_acc:.4f}")
print("### FINISHED CELL 1 ###")


### RUNNING CELL 1: TRAINING ###
Step 1: Configuring paths...
Image data directory: C:\Users\harkp\Desktop\HV-AI-2025\HV-AI-2025\labeled_data\images
Labels CSV path: C:\Users\harkp\Desktop\HV-AI-2025\HV-AI-2025\labeled_data\labeled_data.csv
--------------------------------------------------
Step 2: Defining Custom Dataset and DataLoaders...
Dataset ready. Found 10 classes.
--------------------------------------------------
Step 3: Defining the ResNet-18 model...
Using device: cpu
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\harkp/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:02<00:00, 15.8MB/s]


Model definition complete.
--------------------------------------------------
Step 4: Starting model training...
Epoch 1/10 | Train Loss: 2.0596 | Val Loss: 1.6467 | Val Acc: 0.4936
  -> Validation accuracy improved. Saving model to phase1_baseline_model.pth
Epoch 2/10 | Train Loss: 1.4348 | Val Loss: 1.1177 | Val Acc: 0.6538
  -> Validation accuracy improved. Saving model to phase1_baseline_model.pth
Epoch 3/10 | Train Loss: 1.0799 | Val Loss: 0.8161 | Val Acc: 0.8077
  -> Validation accuracy improved. Saving model to phase1_baseline_model.pth
Epoch 4/10 | Train Loss: 0.8763 | Val Loss: 0.6991 | Val Acc: 0.8141
  -> Validation accuracy improved. Saving model to phase1_baseline_model.pth
Epoch 5/10 | Train Loss: 0.7346 | Val Loss: 0.6213 | Val Acc: 0.8397
  -> Validation accuracy improved. Saving model to phase1_baseline_model.pth
Epoch 6/10 | Train Loss: 0.6594 | Val Loss: 0.5432 | Val Acc: 0.8654
  -> Validation accuracy improved. Saving model to phase1_baseline_model.pth
Epoch 7/10 

In [7]:
### RUNNING CELL 2: TESTING ###

# --------------------------------------------------------------------------
# Step 1: Path Configuration for Testing
# --------------------------------------------------------------------------
# Use the absolute path to your folder containing the test images.
TEST_DIR = r'C:\Users\harkp\Desktop\HV-AI-2025\HV-AI-2025\test_images'

# The model file path remains the same, assuming it's in the same directory as the script.
MODEL_PATH = 'phase1_baseline_model.pth' 

# Define the full path for the output CSV file, including the filename.
OUTPUT_CSV = r'C:\Users\harkp\Desktop\HV-AI-2025\HV-AI-2025\CSV\phase1_predictions.csv'

# Ensure the directory for the output CSV exists
# This line automatically creates the 'CSV to store new predicted csv' folder if it doesn't exist.
output_dir = os.path.dirname(OUTPUT_CSV)
os.makedirs(output_dir, exist_ok=True)

print(f"Test image directory: {TEST_DIR}")
print(f"Model path: {MODEL_PATH}")
print(f"Predictions will be saved to: {OUTPUT_CSV}")
print("-" * 50)

# --------------------------------------------------------------------------
# Step 2: Load Model and Generate Predictions
# --------------------------------------------------------------------------
# (The rest of your code remains the same)
print("Step 2: Loading model and generating predictions...")

# The 'full_dataset' and 'num_classes' variables are available from Cell 1
idx_to_class = full_dataset.idx_to_class

# Initialize the model architecture
prediction_model = models.resnet18(weights=None)
num_ftrs = prediction_model.fc.in_features
prediction_model.fc = nn.Linear(num_ftrs, num_classes)

# Load the saved model weights
try:
    prediction_model.load_state_dict(torch.load(MODEL_PATH))
except FileNotFoundError:
    print(f"ERROR: Model file not found at {MODEL_PATH}")
    print("Please run the training cell (Cell 1) first to generate the model file.")
else:
    prediction_model = prediction_model.to(device)
    prediction_model.eval()

    # Use the validation transforms defined in Cell 1
    test_transforms = data_transforms['val']
    
    predictions = []
    
    test_image_files = [f for f in os.listdir(TEST_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    print(f"Found {len(test_image_files)} images in {TEST_DIR}")

    for filename in test_image_files:
        img_path = os.path.join(TEST_DIR, filename)
        image = Image.open(img_path).convert('RGB')
        image_tensor = test_transforms(image).unsqueeze(0).to(device)

        with torch.no_grad():
            outputs = prediction_model(image_tensor)
            _, predicted_idx = torch.max(outputs, 1)
            predicted_class_name = idx_to_class[predicted_idx.item()]
        
        predictions.append({'path': filename, 'predicted_label': predicted_class_name})
            
    pred_df = pd.DataFrame(predictions)
    pred_df.to_csv(OUTPUT_CSV, index=False)
    print(f"Predictions saved to {OUTPUT_CSV}")
    
    if not pred_df.empty:
        print("\n--- Prediction Results ---")
        print(pred_df.head())
    else:
        print("\nNo test images were found to make predictions on.")
        
print("### FINISHED CELL 2 ###")

Test image directory: C:\Users\harkp\Desktop\HV-AI-2025\HV-AI-2025\test_images
Model path: phase1_baseline_model.pth
Predictions will be saved to: C:\Users\harkp\Desktop\HV-AI-2025\HV-AI-2025\CSV\phase1_predictions.csv
--------------------------------------------------
Step 2: Loading model and generating predictions...
Found 10477 images in C:\Users\harkp\Desktop\HV-AI-2025\HV-AI-2025\test_images
Predictions saved to C:\Users\harkp\Desktop\HV-AI-2025\HV-AI-2025\CSV\phase1_predictions.csv

--- Prediction Results ---
          path predicted_label
0  0_1034.jpeg            cane
1  0_1045.jpeg           ragno
2  0_1085.jpeg           gatto
3   0_110.jpeg            cane
4  0_1104.jpeg           ragno
### FINISHED CELL 2 ###
